In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

/Users/pierrebourbon/Documents/PRO/Data/Master_SISE/NLP/ProjetNLP/projetnlp/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
def infos_resto(url : str) -> pd.DataFrame : 
    '''
    Scrape les informations d'un restaurant à partir d'une URL de base.
    Arguments : 
        - url : une URL de la page web contenant les avis.
    
    Retourne : 
        - Un data frame pandas contenant : 
            - Nom
            - Type de cuisine 
            - Fourchette de prix
            - Adresse 
            - Note moyenne
    '''

    #Ajouter un en-tête User-Agent pour simuler un navigateur
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Cache-Control": "max-age=0",
    }

    #Initialisation 
    nom_resto = []
    type_c = []
    adresse_resto = []
    note_resto = []
    fourchette_resto = []

    #Récupère le contenu HTML de la page concernée
    response = requests.get(url, headers = headers)
    if response.status_code != 200:
        raise Exception(f"Échec de la récupération du contenu, code de statut : {response.status_code}")

    #Parser avec BS4
    fullcorpus = response.text
    soup = BeautifulSoup(fullcorpus, "html.parser")

    ##### NOM DU RESTAURANT ######
    borne_nom = soup.find_all('h1', {'class' : 'biGQs _P egaXP rRtyp'})
    for nom in borne_nom :
        nom_resto.append(nom.text)
    

    ###### TYPE DE CUISINE ######
    borne_tc = soup.find_all('div', {'class' : 'biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W hmDzD'})
    for tc in borne_tc[1] : #Uniquement le deuxième élément qui contient le type de cuisine
        type_c.append(tc.text)

    
    ##### FOURCHETTE DE PRIX #####
    borne_four = soup.find('div', {'class' : 'biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W hmDzD'})
    for four in borne_four :
        fourchette_resto.append(four.text)
    
    #Formatage
    fourchette_resto = [four.replace('\xa0', '') for four in fourchette_resto]


    ##### ADRESSE #####
    borne_adresse = soup.find_all('div', {'class' : 'akmhy e j'})
    for adresse in borne_adresse[0]: #Uniquement le premier élément qui contient l'adresse
        adresse_resto.append(adresse.text)

    
    ##### NOTe MOYENNE #####
    borne_note = soup.find_all('div', {'class' : 'sOyfn u f K'})
    match = re.search(r"(\d+,\d)", borne_note[0].text)
    note_resto = match.group(1)


    #Stocke les résultats dans un dataframe
    results = {
        "Nom" : nom_resto,
        "Type_Cuisine" : type_c,
        "Fourchette_prix" : fourchette_resto,
        "Adresse" : adresse_resto,
        "Note_moyenne" : note_resto
    }

    df_results = pd.DataFrame(results)

    return df_results


In [74]:
test = infos_resto("https://www.tripadvisor.fr/Restaurant_Review-g187265-d695217-Reviews-or15-Brasserie_Georges-Lyon_Rhone_Auvergne_Rhone_Alpes.html")

In [78]:
test.to_csv("infos_resto.csv")

In [ ]:
<a aria-label="Site internet" target="_blank" href="https://frazarin.com/" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS">…</a>

In [ ]:
from selenium import webdriver #Webdriver de Selenium qui permet de contrôler un navigateur
from selenium.webdriver.common.by import By #Permet d'accéder aux différents élements de la page web

from webdriver_manager.chrome import ChromeDriverManager #Assure la gestion du webdriver de Chrome
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

def resto_infos(url : str) -> pd.DataFrame :
    
    # Initialisation du service et du driver
    chrome_options = Options()
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.199 Safari/537.36")
    
    service = Service(ChromeDriverManager().install())

    driver = webdriver.Chrome(service=service, options=chrome_options)

    #Initialisation des listes
    nom_resto = []
    type_c = []
    adresse_resto = []
    note_resto = []
    url_resto = []

    time.sleep(1) #Ajout d'un temps de deux secondes avant de lancer l'action suivante

    #Récupère el contenu HTML de la page
    driver.get(url)

    time.sleep(3) 

    ##### NOM DU RESTAURANT ######
    nom_resto.append(driver.find_element(By.XPATH, "/html/body/div[1]/main/div/div[3]/div/div[1]/h1"))

    # Pause aléatoire entre 3 et 7 secondes
    time.sleep(random.uniform(3, 7))

    return nom_resto

In [46]:
test = resto_infos("https://www.tripadvisor.fr/Restaurant_Review-g187265-d23110895-Reviews-Frazarin-Lyon_Rhone_Auvergne_Rhone_Alpes.html")
print(test)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/main/div/div[3]/div/div[1]/h1"}
  (Session info: chrome=131.0.6778.109); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000107b62e82 chromedriver + 6696578
1   chromedriver                        0x0000000107b5ac9a chromedriver + 6663322
2   chromedriver                        0x0000000107562e3e chromedriver + 405054
3   chromedriver                        0x00000001075b3815 chromedriver + 735253
4   chromedriver                        0x00000001075b3d11 chromedriver + 736529
5   chromedriver                        0x00000001075fe314 chromedriver + 1041172
6   chromedriver                        0x00000001075d889d chromedriver + 886941
7   chromedriver                        0x00000001075fb3f4 chromedriver + 10